In [ ]:
# Setup
import asyncio
from typing import Annotated
from semantic_kernel import Kernel
from semantic_kernel.agents import(
    ChatCompletionAgent, 
    GroupChatOrchestration, 
    RoundRobinGroupChatManager, 
    OrchestrationHandoffs, 
    HandoffOrchestration, 
    ConcurrentOrchestration
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, OpenAIChatCompletion
from semantic_kernel.contents import AuthorRole, ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.functions import kernel_function, KernelArguments

import subprocess
import pandas as pd
import gradio as gr
from datetime import datetime, timedelta
from jaws.jaws_config import *
from jaws.jaws_utils import dbms_connection

# Database not passed, uses the database set in jaws_config.py
driver = dbms_connection(DATABASE)
kernel = Kernel()
settings = OpenAIChatPromptExecutionSettings()
reasoning_service = OpenAIChatCompletion(ai_model_id=OPENAI_REASONING_MODEL, api_key=OPENAI_API_KEY)
kernel.add_service(reasoning_service)
lang_service = OpenAIChatCompletion(ai_model_id=OPENAI_MODEL, api_key=OPENAI_API_KEY)
kernel.add_service(lang_service)

In [2]:
# Tools
class ListInterfaces:
    @kernel_function(description="List available network interfaces. You will never want to select interfaces such as; 'lo' and 'docker0'")
    def list_interfaces(self) -> Annotated[str, "A list of available network interfaces."]:
        interfaces = subprocess.run(['python', './jaws/jaws_capture.py', '--list', '--agent'], capture_output=True, text=True)
        return str(interfaces.stdout)


class CapturePackets:
    @kernel_function(description="Captures packets into the database. Choose a duration depending on the amount of data you want to capture. Recommended not to exceed 60 seconds.")
    def capture_packets(self, interface: str, duration: int) -> Annotated[str, "A process status message once the process is complete."]:
        if duration > 60:
            duration = 60
        packets = subprocess.run(['python', './jaws/jaws_capture.py', '--interface', interface, '--duration', str(duration), '--agent'], capture_output=True, text=True)
        return str(packets.stdout)


class DocumentOrganizations:
    @kernel_function(description="Enriches data with organization ownership by looking up IP addresses.")
    def document_organizations(self) -> Annotated[str, "A process status message once the process is complete."]:
        organizations = subprocess.run(['python', './jaws/jaws_ipinfo.py', '--agent'], capture_output=True, text=True)
        return str(organizations.stdout)


class ComputeEmbeddings:
    @kernel_function(description="Transforms the network traffic data into embeddings for analysis.")
    def compute_embeddings(self) -> Annotated[str, "A process status message once the process is complete."]:
        embeddings = subprocess.run(['python', './jaws/jaws_compute.py', '--agent'], capture_output=True, text=True)
        return str(embeddings.stdout)
    

class AnomalyDetection:
    @kernel_function(description="Analyzes the network traffic data and embeddings and returns a list of anomalies.")
    def anomoly_detection(self) -> Annotated[str, "A string containing a list of anomalies."]:
        output = subprocess.run(['python', './jaws/jaws_finder.py', '--agent'], capture_output=True, text=True)
        return str(output.stdout)
    

class FetchData:
    @kernel_function(description="Fetches the latest (10 minutes) traffic data from the database and returns it as a string.")
    def fetch_traffic(self) -> Annotated[str, "A string containing a list of current traffic data."]:
        query = """
        MATCH (traffic:TRAFFIC)
        WHERE traffic.TIMESTAMP > datetime() - duration({minutes: 10})
        RETURN DISTINCT
            traffic.IP_ADDRESS AS ip_address,
            traffic.PORT AS port,
            traffic.ORGANIZATION AS org,
            traffic.HOSTNAME AS hostname,
            traffic.LOCATION AS location,
            traffic.TOTAL_SIZE AS total_size,
            traffic.OUTLIER AS outlier,
            traffic.TIMESTAMP AS timestamp
        ORDER BY traffic.TIMESTAMP DESC
        LIMIT 100
        """
        with driver.session(database=DATABASE) as session:
            result = session.run(query)
            data = []
            for record in result:
                data.append({
                    'ip_address': record['ip_address'],
                    'port': record['port'],
                    'org': record['org'],
                    'hostname': record['hostname'],
                    'location': record['location'],
                    'total_size': record['total_size'],
                    'outlier': record['outlier'],
                    'timestamp': record['timestamp']
                })
            return str(data)

In [3]:
# Agents
operator_0 = ChatCompletionAgent(
    service=lang_service,
    name="Operator0",
    description="You are the eyes of the network. You are tasked with sampling and reviewing network traffic data to identify patterns, anomalies, escalating to Lead Analyst and their team for further probing and reporting up the chain.",
    instructions=OPERATOR_PROMPT,
    plugins=[ListInterfaces(), CapturePackets(), DocumentOrganizations(), ComputeEmbeddings(), AnomalyDetection()],
    arguments=KernelArguments(settings)
)

operator_1 = ChatCompletionAgent(
    service=lang_service,
    name="Operator1",
    description="You are the eyes of the network. You are tasked with sampling and reviewing network traffic data to identify patterns, anomalies, escalating to Lead Analyst and their team for further probing and reporting up the chain.",
    instructions=OPERATOR_PROMPT,
    plugins=[ListInterfaces(), CapturePackets(), DocumentOrganizations(), ComputeEmbeddings(), AnomalyDetection()],
    arguments=KernelArguments(settings)
)

network_analyst = ChatCompletionAgent(
    service=lang_service, 
    name="NetworkAnalyst",
    description="An expert IT Professional, Sysadmin, and Analyst. Tasked with capturing network packets and performing ETL(Extract, Transform, and Load) with the data to prepare it for analysis.",
    instructions=ANALYST_MANAGED_PROMPT,
    plugins=[ListInterfaces(), CapturePackets(), DocumentOrganizations(), ComputeEmbeddings()],
    arguments=KernelArguments(settings)
)

lead_network_analyst = ChatCompletionAgent(
    service=reasoning_service,
    name="LeadAnalyst",
    description="An expert IT Professional, Sysadmin, and Analyst. Tasked with reviewing network traffic data to identify patterns, anomalies, and make recommendations for security configurations.",
    instructions=MANAGER_PROMPT,
    plugins=[AnomalyDetection(), FetchData()],
    arguments=KernelArguments(settings)
)

handoffs = (
    OrchestrationHandoffs()
    .add(
        source_agent=operator_0.name,
        target_agent=lead_network_analyst.name,
        description="The operator collects short 10-30 second snapshots of network traffic data, analyzes the data for anomalies, and returns a list of red flags to the Lead Analyst.",
    )
    .add(
        source_agent=lead_network_analyst.name,
        target_agent=network_analyst.name,
        description="The Lead Analyst reviews the Operator's list of red flags and requests an additional 30-60 seconds of network traffic data from the Network Analyst to support the Lead Analyst's final report.",
    )
    .add(
        source_agent=network_analyst.name,
        target_agent=lead_network_analyst.name,
        description="The Network Analyst informs the Lead Analyst when the capture and ETL tasks are complete. The Lead Analyst will then review the data and return a moderately detailed report to the command center.",
    )
)

In [4]:
# Orchestration(s)
concurrent_members=[operator_0, operator_1]
handoff_members=[operator_0, lead_network_analyst, network_analyst]
group_members=[network_analyst, lead_network_analyst]
max_rounds = 2


def human_response_function() -> ChatMessageContent:
    message = "No human response required."
    return ChatMessageContent(role=AuthorRole.USER, content=message)


async def agent_callback(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        elif isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        else:
            print(f"{message.name}: {message.content}")


async def concurrent_orchestration(input: str) -> str:
    orchestration_config = ConcurrentOrchestration(members=concurrent_members)
    runtime = InProcessRuntime()
    runtime.start()

    print(f"[INPUT] | {input}")
    
    try:
        print(f"[ORCHESTRATION] | [CONCURRENT] {len(concurrent_members)}")
        result = await orchestration_config.invoke(
            task=input,
            runtime=runtime
        )
        
        response = await result.get()

        response_collection = []
        for item in response:
            response_collection.append(f"{item.name}: {item.content}")

        response_text = str("\n".join(response_collection))
        print(f"[RESPONSE] |\n{response_text}")
        return response_text

    except Exception as e:
        print(f"[ERROR] | {e}")
        return f"[ERROR] | {e}"
        
    finally:
        await runtime.stop_when_idle()
        

async def handoff_orchestration(input: str) -> str:
    orchestration_config = HandoffOrchestration(
        members=handoff_members,
        handoffs=handoffs,
        agent_response_callback=agent_callback,
        human_response_function=human_response_function
    )
    
    runtime = InProcessRuntime()
    runtime.start()

    print(f"[INPUT] | {input}")

    try:
        print(f"[ORCHESTRATION] | [HANDOFF] {len(handoff_members)}")
        result = await orchestration_config.invoke(
            task=input,
            runtime=runtime,
        )
        
        response = await result.get()
        response_text = response.content
        print(f"[RESPONSE] | {response_text}")
        return response_text
       
    except Exception as e:
        print(f"[ERROR] | {e}")
        return f"[ERROR] | {e}"
        
    finally:
        await runtime.stop_when_idle()


async def group_orchestration(input: str) -> str:
    orchestration_config = GroupChatOrchestration(
        members=group_members,
        manager=RoundRobinGroupChatManager(max_rounds=max_rounds),
        agent_response_callback=agent_callback
    )

    runtime = InProcessRuntime()
    runtime.start()

    print(f"[INPUT] | {input}")
    
    try:
        print(f"[ORCHESTRATION] | [GROUP CHAT] {len(group_members)} | [ROUND ROBIN] {max_rounds}")
        result = await orchestration_config.invoke(
            task=input,
            runtime=runtime
        )
        
        response = await result.get()
        response_text = response.content
        print(f"[RESPONSE] | {response_text}")
        return response_text
       
    except Exception as e:
        print(f"[ERROR] | {e}")
        return f"[ERROR] | {e}"
        
    finally:
        await runtime.stop_when_idle()

In [ ]:
# Interface
def next_report(minutes):
    next_report_time = datetime.now() + timedelta(minutes=minutes)
    return f"⏲️ Automatic Report | {minutes} Minutes | {next_report_time.strftime('%H:%M:%S')}"


with gr.Blocks(title="Network Traffic Monitoring") as INTERFACE:
    concurrent_minutes = gr.State(value=15)
    concurrent_seconds = gr.State(value=concurrent_minutes.value * 60)
    concurrent_timer = gr.Timer(value=concurrent_seconds.value, active=True)
    concurrent_chat_history = gr.State(value=[{
        "role": "assistant", 
        "content": "A group of operators tasked with capturing short 10-30 second snapshots of network traffic and returning a list of potential red flags to the command center.",
        "metadata": {"title": "👁️ Traffic Overwatch"}
    }])

    handoff_minutes = gr.State(value=30)
    handoff_seconds = gr.State(value=handoff_minutes.value * 60)
    handoff_timer = gr.Timer(value=handoff_seconds.value, active=True)
    handoff_chat_history = gr.State(value=[{
        "role": "assistant", 
        "content": "A managed group of network analysts tasked with capturing 30-60 second snapshots of network traffic data, enchring the data, and returning a moderately detailed situational report to the command center.",
        "metadata": {"title": "🔎 Traffic Analysis"}
    }])

    groupchat_minutes = gr.State(value=30) # NOT IN USE
    groupchat_seconds = gr.State(value=groupchat_minutes.value * 60)
    groupchat_timer = gr.Timer(value=groupchat_seconds.value, active=True)
    groupchat_history = gr.State(value=[{
        "role": "assistant", 
        "content": "A collaborative group of analysts tasked with capturing 30-60 second snapshots of network traffic data, enchring the data, and returning a comprehensive report for the high command.",
        "metadata": {"title": "🪬 Command Center"}
    }])
    
    with gr.Row(equal_height=True):
        with gr.Column():
            concurrent_chatbot = gr.Chatbot(
                value=concurrent_chat_history.value,
                type="messages",
                show_label=True,
                label="CONCURRENT",
                autoscroll=True,
                resizable=True,
                show_copy_button=True,
                height=480
            )
            concurrent_timer_status = gr.Textbox(
                value=next_report(concurrent_minutes.value),
                #value="No automatic report scheduled.",
                show_label=False,
                container=False,
                interactive=False,
                text_align="center"
            )
            concurrent_request_button = gr.Button("💬 Request Team Report In", variant="huggingface")
    
        with gr.Column():
            handoff_chatbot = gr.Chatbot(
                value=handoff_chat_history.value,
                type="messages",
                show_label=True,
                label="HANDOFF",
                autoscroll=True,
                resizable=True,
                show_copy_button=True,
                height=480
            )
            handoff_timer_status = gr.Textbox(
                value=next_report(handoff_minutes.value),
                #value="No automatic report scheduled.",
                show_label=False,
                container=False,
                interactive=False,
                text_align="center"
            )
            handoff_request_button = gr.Button("💬 Request Team Report In", variant="huggingface")
    
        with gr.Column():
            groupchat_chatbot = gr.Chatbot(
                value=groupchat_history.value,
                type="messages",
                show_label=True,
                label="GROUPCHAT",
                autoscroll=True,
                resizable=True,
                show_copy_button=True,
                height=480
            )
            groupchat_timer_status = gr.Textbox(
                #value=next_report(groupchat_minutes.value),
                value="No automatic report scheduled.",
                show_label=False,
                container=False,
                interactive=False,
                text_align="center"
            )
            groupchat_request_button = gr.Button("💬 Request Team Report In", variant="huggingface")

    async def run_concurrent(history):
        response = await concurrent_orchestration("Perform a short 10-30 second network probe and report back to the command center ASAP.")
        timestamp = datetime.now()
        formatted_response = {"role": "assistant", "content": response, "metadata": {"title": f"️🚩 Red Flag Report | {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"}}
        chat_history = (history + [formatted_response])[-10:]
        return chat_history, chat_history
    
    async def run_handoff(history):
        response = await handoff_orchestration("Perform a 30-60 second network scan and report back the findings to the command center ASAP.")
        timestamp = datetime.now()
        formatted_response = {"role": "assistant", "content": response, "metadata": {"title": f"️📋 Situation Report | {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"}}
        chat_history = (history + [formatted_response])[-3:]
        return chat_history, chat_history
    
    async def run_groupchat(history):
        response = await group_orchestration("High command is requesting the comprehensive network traffic report ASAP.")
        timestamp = datetime.now()
        formatted_response = {"role": "assistant", "content": response, "metadata": {"title": f"🛡️ Briefing for High Command | {timestamp.strftime('%Y-%m-%d %H:%M:%S')}"}}
        chat_history = (history + [formatted_response])[-3:]
        return chat_history, chat_history

    concurrent_timer.tick(
        fn=run_concurrent,
        inputs=[concurrent_chat_history],
        outputs=[concurrent_chatbot, concurrent_chat_history]
    )
    
    concurrent_timer.tick(
        fn=next_report,
        inputs=[concurrent_minutes],
        outputs=[concurrent_timer_status]
    )

    concurrent_request_button.click(
        fn=run_concurrent,
        inputs=[concurrent_chat_history],
        outputs=[concurrent_chatbot, concurrent_chat_history]
    )

    handoff_timer.tick(
        fn=run_handoff,
        inputs=[handoff_chat_history],
        outputs=[handoff_chatbot, handoff_chat_history]
    )
    
    handoff_timer.tick(
        fn=next_report,
        inputs=[handoff_minutes],
        outputs=[handoff_timer_status]
    )

    handoff_request_button.click(
        fn=run_handoff,
        inputs=[handoff_chat_history],
        outputs=[handoff_chatbot, handoff_chat_history]
    )

    """
    groupchat_timer.tick(
        fn=run_groupchat,
        inputs=[groupchat_history],
        outputs=[groupchat_chatbot, groupchat_history]
    )
    
    groupchat_timer.tick(
        fn=next_report,
        inputs=[handoff_minutes],
        outputs=[handoff_timer_status]
    )
    """
    
    groupchat_request_button.click(
        fn=run_groupchat,
        inputs=[groupchat_history],
        outputs=[groupchat_chatbot, groupchat_history]
    )

INTERFACE.launch() #share=True